# Analyses Report Notebook

## Importing necessary packages

In [ ]:
from analyse import HTMLTextAnalysis, WebsiteAnalysis
import pandas

## Create dataframe of complelte article list

In [ ]:
article_list_df = WebsiteAnalysis.get_articles_list_dataframe("../../")

In [ ]:
article_list_df

In [ ]:
article_list_df.sort_values(["latest"], ascending=False).head(10)

In [ ]:
lang_df = article_list_df[article_list_df["language"] == "en"].head(10)
lang_df

## Get all the tokens/words from all the articles

In [ ]:
token_filepath_set = set()
for index, row in article_list_df.iterrows():
    tokens = HTMLTextAnalysis.get_tokens(row["filepath"], lowercase=True, remove_punctuation=True, remove_stopwords=True)
    for token in tokens:
        token_filepath_set.add((token[0], token, row["language"], row["filepath"]))


In [ ]:
tf = pandas.DataFrame(
            token_filepath_set, columns=["first_char", "token", "language", "filepath"]
)


In [ ]:
len(tf["token"].unique())

## Language Statistics 

### Number of articles in each language

In [ ]:
lang_articles = tf[["language", "filepath"]]
lang_articles = lang_articles.drop_duplicates()
lang_articles.groupby(["language"]).count()

In [ ]:
lang_articles_count = lang_articles.groupby(["language"]).count()
lang_articles_count = count.reset_index()

### Number of distinct words in each language

In [ ]:
lang_words = tf[["language", "token"]]
lang_words = lang_words.drop_duplicates()
lang_words_count = lang_words.groupby(["language"]).count()
lang_words_count = lang_words_count.reset_index()
lang_words_count

### Distinct first characters

In [ ]:
first_char_words = tf[["first_char", "token"]]
first_char_words = first_char_words.drop_duplicates()
first_char_words.groupby(["first_char"]).count()

### Overall occurence of words in the articles

Note: Only one occurence in an article is considered

In [ ]:
word_filepath = tf[["token", "first_char", "filepath"]]
word_filepath= word_filepath.drop_duplicates()
word_filepath.groupby(["token"])["token"].count()

In [ ]:
languages ={"en": "English", "fr": "French", "ml": "Malayalam", "pa": "Punjabi", "hi": "Hindi"}
with open("../../templates/report.html", "r") as blogtemplate:
    content = blogtemplate.read()
    for code, name in languages.items():
        content = content.replace(name + "Articles", str(lang_articles_count[lang_articles_count["language"] == code]["filepath"].values[0]))
    for code, name in languages.items():
        content = content.replace(name + "Words", str(lang_words_count[lang_words_count["language"] == code]["token"].values[0]))
    with open("../../blog/report.html", "w") as blog:
        blog.write(content)
    blog.close()
blogtemplate.close()


In [ ]:
tf = tf.sort_values(["first_char", "token"])
tf.to_csv("test.csv", index=False)

In [ ]:
word_filepath_list = word_filepath.sort_values(["first_char", "token"])
word_filepath_list.to_html("test.html", index=False)